# Attention Architecture

In this notebook we are going to dive into the Attention mechanism in Sequence2Sequence models to understand how they work and can be implemented in PyTorch.

This is needed so we can later, in another Notebook, understand the Transformer Model and apply it correctly.
For reading up on the Attention mechanism, I propose [this blog article](https://jalammar.github.io/visualizing-neural-machine-translation-mechanics-of-seq2seq-models-with-attention/) which explains the concept in a visual yet detailed way.
Additionally, the PyTorch Tutorials also give a good explanation on the subject: [Link](https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html#training)

In [1]:
import pandas as pd
# import sys
# sys.path.insert(0, '.') # searches this directory too -> ensures that imports work fine.

# from modules.tatoeba import TatoebaDataset
from modules.tokenizer import Tokenizer

In [2]:
from modules.tatoeba import TatoebaDataset

In [4]:
data = pd.read_csv('data/deu-eng/deu.txt', sep="\t", names=['en', 'de', 'license'])
ttds = TatoebaDataset(data, max_length=10)

In [17]:
en, de = ttds[0]

## The Encoder Architecture

The encoder will take our "word-ids" (remember the Tokenizer) and encode them into a a vector: The hidden representation. Lateron our decoder network will use this hidden representation to generate the needed output.